# Analyze results
Now that we have some results for the runs, we would like to know whether a regression algorithm is statistically better than the others, for each task. We will need the separate predictions for each task. ChatGPT tells me that, since all folds are the same for all algorithms, the best statistical test to compare them is a *paired* one. Since there are more than two algorithms, we should use the Friedman test, or so the mighty oracle says, followed by a Wilcoxon signed-rank test. Wow, that's a lot of important-sounding names

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re as regex
import seaborn as sns

from itertools import combinations
from scipy.stats import friedmanchisquare, wilcoxon
from sklearn.metrics import r2_score

In [23]:
# some hard-coded parameters for the analysis
results_folder = "../results_server" # VSCODE likes to use the root folder as the working directory, but NOT for notebooks
task_details_file = "../results_server/20250722_openml_ctr23_statistics_mu50.csv"

significance_level = 0.05 # arbitrary, could also be 0.01

In [ ]:
# first, let's list all *csv files in the results folder; and select the ones with the correct pattern
results_files = [
    f for f in os.listdir(results_folder) if regex.match(r"^\w+_task_\d+_fold_\d+\.csv$", f)
    and os.path.isfile(os.path.join(results_folder, f))
]
print(f"Found {len(results_files)} results files: {results_files}")

# then, let's identify the unique tasks and algorithms from the filenames
tasks = set()
algorithms = set()
for file in results_files:
    match = regex.match(r"(\w+)_task_(\d+)_fold_(\d+)\.csv", file)
    if match:
        algorithm, task, _ = match.groups()
        algorithms.add(algorithm)
        tasks.add(task)

# sort the algorithms and the tasks
algorithms = sorted(list(algorithms))
tasks = sorted(list([int(t) for t in tasks]))

print(f"Found {len(tasks)} tasks: {tasks}")
print(f"Found {len(algorithms)} algorithms: {algorithms}")

Found 1050 results files: ['PySRRegressor_task_361234_fold_0.csv', 'PySRRegressor_task_361234_fold_1.csv', 'PySRRegressor_task_361234_fold_2.csv', 'PySRRegressor_task_361234_fold_3.csv', 'PySRRegressor_task_361234_fold_4.csv', 'PySRRegressor_task_361234_fold_5.csv', 'PySRRegressor_task_361234_fold_6.csv', 'PySRRegressor_task_361234_fold_7.csv', 'PySRRegressor_task_361234_fold_8.csv', 'PySRRegressor_task_361234_fold_9.csv', 'PySRRegressor_task_361235_fold_0.csv', 'PySRRegressor_task_361235_fold_1.csv', 'PySRRegressor_task_361235_fold_2.csv', 'PySRRegressor_task_361235_fold_3.csv', 'PySRRegressor_task_361235_fold_4.csv', 'PySRRegressor_task_361235_fold_5.csv', 'PySRRegressor_task_361235_fold_6.csv', 'PySRRegressor_task_361235_fold_7.csv', 'PySRRegressor_task_361235_fold_8.csv', 'PySRRegressor_task_361235_fold_9.csv', 'PySRRegressor_task_361236_fold_0.csv', 'PySRRegressor_task_361236_fold_1.csv', 'PySRRegressor_task_361236_fold_2.csv', 'PySRRegressor_task_361236_fold_3.csv', 'PySRRegresso

In [25]:
# now, let's go task by task, and computer the R2 score for each algorithm from the predictions
results = {task: {alg: [] for alg in algorithms} for task in tasks}
r2_results = {task: {alg: {'mean': None, 'stdev' : None} for alg in algorithms} for task in tasks}
significant_best_algorithms = {task: [] for task in tasks}

for task in tasks:
    for algorithm in algorithms :
        # find the files for this task and algorithm
        task_files = [f for f in results_files if f.startswith(f"{algorithm}_task_{str(task)}_")]
        
        # read the predictions from each file and compute the R2 score
        for file in task_files :
            df = pd.read_csv(os.path.join(results_folder, file))
            column_true = [c for c in df.columns if c.endswith('_true')][0]
            column_pred = [c for c in df.columns if c.endswith('_pred')][0]
            
            # no error control, because we assume the files are well-formed
            r2 = r2_score(df[column_true], df[column_pred])
            results[task][algorithm].append(r2)

        r2_results[task][algorithm]['mean'] = np.mean(results[task][algorithm])
        r2_results[task][algorithm]['stdev'] = np.std(results[task][algorithm])

    # now that we have all the R2 scores for this task, we can compute the Friedman test
    r2_scores = [results[task][alg] for alg in algorithms]
    friedman_stat, p_value = friedmanchisquare(*r2_scores)

    if p_value < significance_level :
        print(f"Task {task}: Friedman test significant (p-value = {p_value:.4e})")
        
        # perform pairwise Wilcoxon signed-rank tests
        for alg1, alg2 in combinations(algorithms, 2):
            stat, p = wilcoxon(results[task][alg1], results[task][alg2])
            if p < significance_level:
                print(f"  {alg1} vs {alg2}: Wilcoxon test significant (p-value = {p:.4f})")
                if r2_results[task][alg1]['mean'] > r2_results[task][alg2]['mean'] :
                    significant_best_algorithms[task].append(alg1)
                elif r2_results[task][alg1]['mean'] < r2_results[task][alg2]['mean'] :
                    significant_best_algorithms[task].append(alg2)
            else :
                print(f"  {alg1} vs {alg2}: Wilcoxon test NOT significant (p-value = {p:.4f})")
                if max([r2_results[task][a]['mean'] for a in algorithms]) == r2_results[task][alg1]['mean'] or \
                     max([r2_results[task][a]['mean'] for a in algorithms]) == r2_results[task][alg2]['mean'] :
                    significant_best_algorithms[task].append(alg1)
                    significant_best_algorithms[task].append(alg2)
        # remove duplicates
        significant_best_algorithms[task] = list(set(significant_best_algorithms[task]))
    else :
        print(f"Task {task}: Friedman test not significant (p-value = {p_value:.4f})")

Task 361234: Friedman test significant (p-value = 1.1167e-04)
  PySRRegressor vs RandomForestRegressor: Wilcoxon test significant (p-value = 0.0059)
  PySRRegressor vs XGBRegressor: Wilcoxon test significant (p-value = 0.0020)
  RandomForestRegressor vs XGBRegressor: Wilcoxon test significant (p-value = 0.0020)
Task 361235: Friedman test significant (p-value = 1.1167e-04)
  PySRRegressor vs RandomForestRegressor: Wilcoxon test significant (p-value = 0.0020)
  PySRRegressor vs XGBRegressor: Wilcoxon test significant (p-value = 0.0020)
  RandomForestRegressor vs XGBRegressor: Wilcoxon test significant (p-value = 0.0039)
Task 361236: Friedman test significant (p-value = 4.5400e-05)
  PySRRegressor vs RandomForestRegressor: Wilcoxon test significant (p-value = 0.0020)
  PySRRegressor vs XGBRegressor: Wilcoxon test significant (p-value = 0.0020)
  RandomForestRegressor vs XGBRegressor: Wilcoxon test significant (p-value = 0.0020)
Task 361237: Friedman test significant (p-value = 4.5400e-05)

In [26]:
# and now we can create a latex table with the results, highlighting the significant best algorithms
print("\n\\begin{tabular}{|c|c|c|c|}")
print("\\hline")
print("Task & Algorithm & Mean R2 & Stdev R2 \\\\")
print("\\hline")
for task in tasks:
    for algorithm in algorithms:
        mean_r2 = r2_results[task][algorithm]['mean']
        stdev_r2 = r2_results[task][algorithm]['stdev']
        if algorithm in significant_best_algorithms[task]:
            print(f"{task} & {algorithm} & \\textbf{{{mean_r2:.4f}}}* & \\textbf{stdev_r2:.4f} \\\\")
        else:
            print(f"{task} & {algorithm} & {mean_r2:.4f} & {stdev_r2:.4f} \\\\")
    print("\\hline")
print("\\hline")
print("\\end{tabular}")



\begin{tabular}{|c|c|c|c|}
\hline
Task & Algorithm & Mean R2 & Stdev R2 \\
\hline
361234 & PySRRegressor & \textbf{0.5698}* & \textbf0.0224 \\
361234 & RandomForestRegressor & \textbf{0.5490}* & \textbf0.0173 \\
361234 & XGBRegressor & 0.4586 & 0.0264 \\
\hline
361235 & PySRRegressor & 0.6581 & 0.0506 \\
361235 & RandomForestRegressor & \textbf{0.9415}* & \textbf0.0125 \\
361235 & XGBRegressor & \textbf{0.9608}* & \textbf0.0099 \\
\hline
361236 & PySRRegressor & 0.8824 & 0.0289 \\
361236 & RandomForestRegressor & \textbf{0.9946}* & \textbf0.0016 \\
361236 & XGBRegressor & \textbf{0.9992}* & \textbf0.0002 \\
\hline
361237 & PySRRegressor & 0.8301 & 0.0335 \\
361237 & RandomForestRegressor & \textbf{0.9150}* & \textbf0.0272 \\
361237 & XGBRegressor & \textbf{0.9365}* & \textbf0.0235 \\
\hline
361241 & PySRRegressor & 0.3276 & 0.0142 \\
361241 & RandomForestRegressor & \textbf{0.6819}* & \textbf0.0045 \\
361241 & XGBRegressor & \textbf{0.6474}* & \textbf0.0042 \\
\hline
361242 & PySRRegr

In [ ]:
# ok, but actually what about a nice Excel spreadsheet?
# first, let's read a file with all the task details
df_tasks = pd.read_csv(task_details_file)

# and now we are going to create a dictionary that will be later converted to a DataFrame
results_dict = {
    'task_id' : [],
    'dataset_name' : [],
    'target_name' : [],
    'n_samples' : [],
    'n_features' : [],
    'missing_data' : [],
    'categorical_features' : [],
    'R2_PySRRegressor' : [],
    'R2_RandomForestRegressor' : [],
    'R2_XGBRegressor' : [],
}

# copy all the other columns from the df_tasks DataFrame
for column in results_dict.keys() :
    if not column.startswith('R2_') :
        results_dict[column] = df_tasks[column].tolist()

# rewrite the R2 scores for each algorithm
for task in tasks:
    for algorithm in algorithms:

        results_dict[f'R2_{algorithm}'].append("%.4f +/- %.4f" %
                                               (r2_results[task][algorithm]['mean'], r2_results[task][algorithm]['stdev']))
        
# and now we can create a DataFrame from the dictionary
df_results = pd.DataFrame(results_dict)

# color the cells, marking the significant best algorithms for each task
def highlight_best_algorithms(row) :
    task = row['task_id']
    best_algorithms = significant_best_algorithms.get(task, [])
    # col[3:] is used to skip the 'R2_' prefix, so it's the name of the algorithm
    # #e6ffe6 is a very light green color
    return [
        'background-color: #e6ffe6' if col.startswith('R2_') and col[3:] in best_algorithms else ''
        for col in row.index
    ]

# apply the highlighting function to the R2 columns
r2_columns = [f'R2_{alg}' for alg in algorithms]
df_results = df_results.style.apply(highlight_best_algorithms, axis=1)

# finally, save to Excel
df_results.to_excel("../results_server/analysis_openml_ctr23_statistics_mu50_results.xlsx", index=False)